In [1]:
import re
import copy
import math
import pickle
import numpy
import matplotlib.pyplot as plt
import urllib
import nltk
import scipy.optimize
import random
from collections import defaultdict
import string

In [2]:
from nltk.stem.porter import*
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
# read data (already shuffle)
f_train = open('movie_train_data_item', 'rb')
train_data = pickle.load(f_train)
f_train.close()
f_test = open('movie_test_data_item', 'rb')
test_data = pickle.load(f_test)
f_test.close()

In [4]:
# use 300000 data in consideration of calculation time
# train_data=train_data[:300000]
# test_data=test_data[:300000]

In [5]:
# delete abnormal data whose helpful ratio > 1
for datum in train_data:
    if datum['helpful'][1] != 0 and float(datum['helpful'][0])/datum['helpful'][1] > 1:
        print datum['helpful']
        train_data.remove(datum)
for datum in test_data:
    if datum['helpful'][1] != 0 and float(datum['helpful'][0])/datum['helpful'][1] > 1:
        print datum['helpful']
        test_data.remove(datum)

In [6]:
print len(train_data)
print len(test_data)

255330
45058


In [7]:
# look for data whose helpful is not [0,0]
non_zero_train_data = []
for datum in train_data:
    if datum['helpful'][1] != 0:
        non_zero_train_data.append(datum)
non_zero_test_data = []
for datum in test_data:
    if datum['helpful'][1] != 0:
        non_zero_test_data.append(datum)

In [8]:
# filter data for 0 < helpful ratio < 0.2 and 0.8 < helpful ratio < 1.0
filtered_train_data=[]
for datum in non_zero_train_data:
    if float(datum['helpful'][0])/datum['helpful'][1] < 1 and float(datum['helpful'][0])/datum['helpful'][1] > 0.8:
        filtered_train_data.append(datum)
    if float(datum['helpful'][0])/datum['helpful'][1] < 0.2 and float(datum['helpful'][0])/datum['helpful'][1] > 0:
        filtered_train_data.append(datum)

In [9]:
# filter data for 0 < helpful ratio < 0.2 and 0.8 < helpful ratio < 1.0
filtered_test_data=[]
for datum in non_zero_test_data:
    if float(datum['helpful'][0])/datum['helpful'][1] < 1 and float(datum['helpful'][0])/datum['helpful'][1] > 0.8:
        filtered_test_data.append(datum)
    if float(datum['helpful'][0])/datum['helpful'][1] < 0.2 and float(datum['helpful'][0])/datum['helpful'][1] > 0:
        filtered_test_data.append(datum)

In [10]:
len(filtered_train_data)

23731

In [11]:
len(filtered_test_data)

23731

In [7]:
filtered_train_data = copy.deepcopy(train_data)
filtered_test_data = copy.deepcopy(test_data)

In [8]:
# preprocess (without stem)
processed_review_no_stem = []
stop_words = set(stopwords.words('english'))
self_defined_stop_words=set([''])
count=0
for datum in filtered_train_data:
    count+=1
    if count % 1000 == 0:
        print count
    ### Ignore capitalization and remove punctuation
    wordCount = defaultdict(int)
    punctuation = set(string.punctuation)
    r = ''.join([c for c in datum['reviewText'].lower() if not c in punctuation])
    ### Remove Number
    r_no_number = re.sub('[0-9]+', '', r)
    word_tokens = word_tokenize(r_no_number)
    ## Remove stop words
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    for word in filtered_sentence:
        if len(word) < 3:
            filtered_sentence.remove(word)
    processed_review_no_stem.append(filtered_sentence)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [20]:
### Stem
processed_review_words_stem = []
stemmer = PorterStemmer()
count=0
for i in range(len(processed_review_no_stem)):
    count+=1
    if count % 1000 == 0:
        print count
    processed_review_words_stem.append([stemmer.stem(w) for w in processed_review_no_stem[i]])

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000


In [9]:
# get number of sentence
num_sentence = []
for datum in filtered_train_data:
    sentence_token = nltk.sent_tokenize(datum['reviewText'])
    for token in sentence_token:
        if len(token) <= 2:
            sentence_token.remove(token)
    num_sentence.append(len(sentence_token))

In [10]:
# get number of characters
num_character = []
for datum in processed_review_no_stem:
    num_character.append(sum([len(w) for w in datum]))

In [41]:
# get number of complex words ( syllables >= 3)
# method 1
def sylco(word):
    word = word.lower()
    # exception_add are words that need extra syllables
    # exception_del are words that need less syllables
    exception_add = ['serious','crucial']
    exception_del = ['fortunately','unfortunately']
    co_one = ['cool','coach','coat','coal','count','coin','coarse','coup','coif','cook','coign','coiffe','coof','court']
    co_two = ['coapt','coed','coinci']
    pre_one = ['preach']
    syls = 0 #added syllable number
    disc = 0 #discarded syllable number
    #1) if letters < 3 : return 1
    if len(word) <= 3 :
        syls = 1
        return syls
    #2) if doesn't end with "ted" or "tes" or "ses" or "ied" or "ies", discard "es" and "ed" at the end.
    # if it has only 1 vowel or 1 set of consecutive vowels, discard. (like "speed", "fled" etc.)
    if word[-2:] == "es" or word[-2:] == "ed" :
        doubleAndtripple_1 = len(re.findall(r'[eaoui][eaoui]',word))
        if doubleAndtripple_1 > 1 or len(re.findall(r'[eaoui][^eaoui]',word)) > 1 :
            if word[-3:] == "ted" or word[-3:] == "tes" or word[-3:] == "ses" or word[-3:] == "ied" or word[-3:] == "ies" :
                pass
            else :
                disc+=1
    #3) discard trailing "e", except where ending is "le"  
    le_except = ['whole','mobile','pole','male','female','hale','pale','tale','sale','aisle','whale','while']
    if word[-1:] == "e" :
        if word[-2:] == "le" and word not in le_except :
            pass
        else :
            disc+=1
    #4) check if consecutive vowels exists, triplets or pairs, count them as one.
    doubleAndtripple = len(re.findall(r'[eaoui][eaoui]',word))
    tripple = len(re.findall(r'[eaoui][eaoui][eaoui]',word))
    disc+=doubleAndtripple + tripple
    #5) count remaining vowels in word.
    numVowels = len(re.findall(r'[eaoui]',word))
    #6) add one if starts with "mc"
    if word[:2] == "mc" :
        syls+=1
    #7) add one if ends with "y" but is not surrouned by vowel
    if word[-1:] == "y" and word[-2] not in "aeoui" :
        syls +=1
    #8) add one if "y" is surrounded by non-vowels and is not in the last word.
    for i,j in enumerate(word) :
        if j == "y" :
            if (i != 0) and (i != len(word)-1) :
                if word[i-1] not in "aeoui" and word[i+1] not in "aeoui" :
                    syls+=1
    #9) if starts with "tri-" or "bi-" and is followed by a vowel, add one.
    if word[:3] == "tri" and word[3] in "aeoui" :
        syls+=1
    if word[:2] == "bi" and word[2] in "aeoui" :
        syls+=1
    #10) if ends with "-ian", should be counted as two syllables, except for "-tian" and "-cian"
    if word[-3:] == "ian" : 
    #and (word[-4:] != "cian" or word[-4:] != "tian") :
        if word[-4:] == "cian" or word[-4:] == "tian" :
            pass
        else :
            syls+=1
    #11) if starts with "co-" and is followed by a vowel, check if exists in the double syllable dictionary, if not, check if in single dictionary and act accordingly.
    if word[:2] == "co" and word[2] in 'eaoui' :
 
        if word[:4] in co_two or word[:5] in co_two or word[:6] in co_two :
            syls+=1
        elif word[:4] in co_one or word[:5] in co_one or word[:6] in co_one :
            pass
        else :
            syls+=1
    #12) if starts with "pre-" and is followed by a vowel, check if exists in the double syllable dictionary, if not, check if in single dictionary and act accordingly.
    if word[:3] == "pre" and word[3] in 'eaoui' :
        if word[:6] in pre_one :
            pass
        else :
            syls+=1
    #13) check for "-n't" and cross match with dictionary to add syllable.
    negative = ["doesn't", "isn't", "shouldn't", "couldn't","wouldn't"]
    if word[-3:] == "n't" :
        if word in negative :
            syls+=1
        else :
            pass  
    #14) Handling the exceptional words.
    if word in exception_del :
        disc+=1
    if word in exception_add :
        syls+=1    
    # calculate the output
    return numVowels - disc + syls
################################################
num_complex_word = []
count=0
for datum in processed_review_no_stem:
    count+=1
    if count % 1000 == 0:
        print count
    num_complex_word.append(sum([1 for w in datum if sylco(w)>=3]))

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000


In [11]:
#method 2
def syllables(word):
    #referred from stackoverflow.com/questions/14541303/count-the-number-of-syllables-in-a-word
    count = 0
    vowels = 'aeiouy'
    word = word.lower()
    if word[0] in vowels:
        count +=1
    for index in range(1,len(word)):
        if word[index] in vowels and word[index-1] not in vowels:
            count +=1
    if word.endswith('e'):
        count -= 1
    if word.endswith('le'):
        count+=1
    if count == 0:
        count +=1
    return count
################################################
num_complex_word = []
count=0
for datum in processed_review_no_stem:
    count+=1
    if count % 1000 == 0:
        print count
    num_complex_word.append(sum([1 for w in datum if syllables(w)>=3]))

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [71]:
print processed_review_no_stem[2]
for w in processed_review_no_stem[2]:
    print syllables(w)
print '#################'
print num_complex_word[2]

['one', 'dumbest', 'shows', 'ever', 'hoosiers', 'terrible', 'suck', 'even', 'thought', 'animated', 'would', 'wish', 'worst', 'enemy']
1
2
1
2
2
3
1
2
1
4
1
1
1
3
#################
3


In [12]:
# Readability
### Evaluating content quality and helpfulness of online product reviews: The interplay of review helpfulness vs. review content

# The Gunning-Fog Index (FOG)
FOG=[]
for i in range(len(num_sentence)):
    if num_sentence[i]==0:
        num_sentence[i] += 1
for i in range(len(filtered_train_data)):
    if len(processed_review_no_stem[i])==0:
        FOG.append(0.0)
    else:
        FOG.append(0.4*((float(len(processed_review_no_stem[i]))/num_sentence[i])+float(num_complex_word[i]*100)/len(processed_review_no_stem[i]))) 

In [13]:
print FOG[0]

12.5263681592


In [14]:
# The Flesch–Kincaid Reading Ease (FK)
FK = []
for i in range(len(num_sentence)):
    if num_sentence[i]==0:
        num_sentence[i] += 1
for i in range(len(filtered_train_data)):
    if len(processed_review_no_stem[i])==0:
        FK.append(0.0)
    else:
        FK.append(0.39*(float(len(processed_review_no_stem[i]))/num_sentence[i])+float(num_complex_word[i]*11.8)/len(processed_review_no_stem[i])-15.59)

In [15]:
print FK[0]

-8.8573880597


In [16]:
# The Automated Readability Index (ARI)
ARI = []
for i in range(len(filtered_train_data)):
    if len(processed_review_no_stem[i])==0:
        ARI.append(0.0) 
    else:
        ARI.append(4.71*(float(num_character[i])/len(processed_review_no_stem[i]))+0.5*(float(len(processed_review_no_stem[i]))/num_sentence[i])-21.43)

In [17]:
print ARI[0]

11.0425124378


In [18]:
# The Coleman-LiauIndex (CLI)
CLI = []
for i in range(len(filtered_train_data)):
    if len(processed_review_no_stem[i])==0:
        CLI.append(0.0) 
    else:
        CLI.append(5.89*(float(num_character[i])/len(processed_review_no_stem[i]))+0.3*(float(num_sentence[i])/len(processed_review_no_stem[i]))-15.8)

In [19]:
print CLI[0]

17.8526119403


In [24]:
# Information Complexity
# Classify word type
# get all types of words
word_types = []
count=0
for datum in processed_review_no_stem:
    count+=1
    if count % 1000 ==0:
        print count
    for word_tag in nltk.pos_tag(datum):
        if not word_tag[1] in word_types:
            word_types.append(word_tag[1])

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [25]:
print word_types

['RB', 'VBD', 'JJ', 'NN', 'NNS', 'CD', 'VBP', 'VBG', 'IN', 'NNP', 'VBN', 'VBZ', 'CC', 'RBR', 'VB', 'RP', 'JJR', 'MD', 'JJS', 'DT', 'FW', 'RBS', 'PRP$', 'WP', 'WP$', 'WDT', 'EX', 'TO', 'PRP', 'UH', 'WRB', 'PDT', 'POS', 'NNPS', "''", '$', '``', 'SYM']


In [20]:
word_types = ['RB', 'VBD', 'JJ', 'NN', 'NNS', 'CD', 'VBP', 'VBG', 'IN', 'NNP', 'VBN', 'VBZ', 'CC', 'RBR', 'VB', 'RP', 'JJR', 'MD', 'JJS', 'DT', 'FW', 'RBS', 'PRP$', 'WP', 'WP$', 'WDT', 'EX', 'TO', 'PRP', 'UH', 'WRB', 'PDT', 'POS', 'NNPS', "''", '$', '``', 'SYM']

In [21]:
# Convert comparative and superlative to normal
processed_review_no_stem_normalized=[]
wnl = WordNetLemmatizer()
count=0
for datum in processed_review_no_stem:
    count+=1
    if count % 1000 ==0:
        print count
    tmp_word_list = []
    for word in datum:
        word = wnl.lemmatize(word, 'n')
        word = wnl.lemmatize(word, 'v')
        word = wnl.lemmatize(word, 'a')
        word = wnl.lemmatize(word, 's')
        word = wnl.lemmatize(word, 'r')
        tmp_word_list.append(word)
    processed_review_no_stem_normalized.append(tmp_word_list)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [22]:
# look for 20 most frequent normalized words
words_normalized_count={}
for text in processed_review_no_stem_normalized:
    for word in text:
        if not words_normalized_count.has_key(word):
            words_normalized_count[word] = 0
        words_normalized_count[word] += 1

In [23]:
words_normarlized_most_frequent = sorted(words_normalized_count.iteritems(), key=lambda d:d[1], reverse = True)
words_normarlized_most_frequent = [word[0] for word in words_normarlized_most_frequent[:20]]

In [24]:
print words_normarlized_most_frequent

['movie', 'film', 'one', 'good', 'like', u'make', 'see', 'get', u'great', u'watch', 'time', 'story', 'character', 'love', u'show', 'would', u'think', 'really', 'well', 'first']


In [25]:
# Information Complexity
# For normalized word not in top-20
tag_count = []
num_types = len(word_types)
count=0
for datum in processed_review_no_stem_normalized:
    count+=1
    if count % 1000 ==0:
        print count
    tags = nltk.pos_tag(datum)
    tag_num_dict = {}
    for i in range(num_types):
        tag_num_dict[word_types[i]]=sum([1 for t in tags if t[1]==word_types[i] and not t[0] in words_normarlized_most_frequent])
    tag_count.append(tag_num_dict)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [26]:
print processed_review_no_stem[1]
print processed_review_no_stem_normalized[1]

['finished', 'looking', 'quotayn', 'rand', 'sense', 'lifequot', 'must', 'say', 'happy', 'found', 'dvd', 'insightful', 'portrayal', 'ayn', 'interviews', 'ayn', 'discussing', 'philosophy', 'objectivism', 'priceless', 'wonderful', 'way', 'learn', 'also', 'story', 'many', 'scenes', 'old', 'new', 'york', 'exciting', 'say', 'want', 'know', 'rand', 'buy', 'dvd', 'sorrydiane', 'offuttatlanta', 'georgia']
[u'finish', u'look', 'quotayn', 'rand', 'sense', 'lifequot', 'must', 'say', 'happy', u'find', 'dvd', 'insightful', 'portrayal', 'ayn', u'interview', 'ayn', u'discuss', 'philosophy', 'objectivism', 'priceless', 'wonderful', 'way', 'learn', 'also', 'story', 'many', u'scene', 'old', 'new', 'york', u'excite', 'say', 'want', 'know', 'rand', 'buy', 'dvd', 'sorrydiane', 'offuttatlanta', 'georgia']


In [27]:
print tag_count[1]

{'PRP$': 0, 'VBG': 0, 'FW': 0, '``': 0, 'VBN': 0, "''": 0, 'VBP': 4, 'WDT': 0, 'JJ': 13, 'WP': 0, 'VBZ': 0, 'DT': 0, 'RP': 0, '$': 0, 'NN': 17, 'VBD': 0, 'POS': 0, 'TO': 0, 'PRP': 0, 'RB': 1, 'NNS': 0, 'NNP': 0, 'VB': 3, 'WRB': 0, 'CC': 0, 'PDT': 0, 'RBS': 0, 'RBR': 0, 'CD': 0, 'EX': 0, 'IN': 0, 'WP$': 0, 'MD': 1, 'NNPS': 0, 'JJS': 0, 'JJR': 0, 'SYM': 0, 'UH': 0}


In [28]:
# fraction score
fraction_noun = []
fraction_adj = []
fraction_verb = []
for datum in tag_count:
    fraction_noun.append(float(datum['NN']+datum['NNS']+datum['NNP'])/num_sentence[i])
    fraction_adj.append(float(datum['JJ']+datum['JJS']+datum['JJR'])/num_sentence[i])
    fraction_verb.append(float(datum['VB']+datum['VBD']+datum['VBG']+datum['VBN']+datum['VBP']+datum['VBZ'])/num_sentence[i])

In [29]:
print fraction_noun[2]

2.5


In [30]:
print processed_review_no_stem_normalized[1]

[u'finish', u'look', 'quotayn', 'rand', 'sense', 'lifequot', 'must', 'say', 'happy', u'find', 'dvd', 'insightful', 'portrayal', 'ayn', u'interview', 'ayn', u'discuss', 'philosophy', 'objectivism', 'priceless', 'wonderful', 'way', 'learn', 'also', 'story', 'many', u'scene', 'old', 'new', 'york', u'excite', 'say', 'want', 'know', 'rand', 'buy', 'dvd', 'sorrydiane', 'offuttatlanta', 'georgia']


In [31]:
train_text_mining_feature = numpy.zeros((len(train_data),8))
index=0
#for each row
# b f_noun f_adj f_verb FOG FK ARI CLI
for datum in train_data:
    train_text_mining_feature[index]=[datum['helpful'][1],fraction_noun[index],fraction_adj[index],fraction_verb[index],FOG[index],FK[index],ARI[index],CLI[index]]
    index += 1
    if index % 1000 == 0:
        print index

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [32]:
print train_text_mining_feature[5]

[  0.           5.           2.           1.5          9.86666667
 -10.50333333  11.02625     19.82291667]


In [33]:
# output
import pickle
data_np_array = open('train_text_feature','wb')
pickle.dump(train_text_mining_feature,data_np_array)
data_np_array.close()

In [21]:
# output
import pickle
dataarray = open('tag_count_no_stem','wb')
pickle.dump(tag_count,dataarray)
dataarray.close()

In [144]:
# count uppercase, lowercase, exclamation and period
upper_count=[]
lower_count=[]
exclamation_count=[]
period_count=[]
for datum in filtered_train_data:
    upper_count.append(sum([1 for c in datum['reviewText'] if c.isupper()]))
    lower_count.append(sum([1 for c in datum['reviewText'] if c.islower()]))
    exclamation_count.append(sum([1 for c in datum['reviewText'] if c=='!']))
    period_count.append(sum([1 for c in datum['reviewText'] if c=='.']))

In [143]:
print filtered_train_data[1]
print upper_count[1]
print exclamation_count[1]
print period_count[1]

{'reviewerID': 'A2J57EQ9KPGXR9', 'asin': 'B00005OW0H', 'helpful': [1, 26], 'reviewText': 'What horrible animation! Overly glossy, computer generated garbage. And the &quot;story&quot;, just plain awful. I could actually feel my brain shrinking as I watched this mess. I find it hard to believe that kids over the age of 4 enjoy this mindless nonsense and anyone else would voluntarily watch this Pokemess.Blech!!', 'overall': 1.0, 'unixReviewTime': 1003795200}
8
3
4


# Feature for Test Set

In [34]:
# preprocess (without stem)
test_processed_review_no_stem = []
stop_words = set(stopwords.words('english'))
self_defined_stop_words=set([''])
count=0
for datum in test_data:
    count+=1
    if count % 1000 == 0:
        print count
    ### Ignore capitalization and remove punctuation
    wordCount = defaultdict(int)
    punctuation = set(string.punctuation)
    r = ''.join([c for c in datum['reviewText'].lower() if not c in punctuation])
    ### Remove Number
    r_no_number = re.sub('[0-9]+', '', r)
    word_tokens = word_tokenize(r_no_number)
    ## Remove stop words
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    for word in filtered_sentence:
        if len(word) < 3:
            filtered_sentence.remove(word)
    test_processed_review_no_stem.append(filtered_sentence)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000


In [35]:
# get number of sentence
test_num_sentence = []
for datum in test_data:
    sentence_token = nltk.sent_tokenize(datum['reviewText'])
    for token in sentence_token:
        if len(token) <= 2:
            sentence_token.remove(token)
    test_num_sentence.append(len(sentence_token))

In [36]:
# get number of characters
test_num_character = []
for datum in test_processed_review_no_stem:
    test_num_character.append(sum([len(w) for w in datum]))

In [37]:
# count syllables
def syllables(word):
    #referred from stackoverflow.com/questions/14541303/count-the-number-of-syllables-in-a-word
    count = 0
    vowels = 'aeiouy'
    word = word.lower()
    if word[0] in vowels:
        count +=1
    for index in range(1,len(word)):
        if word[index] in vowels and word[index-1] not in vowels:
            count +=1
    if word.endswith('e'):
        count -= 1
    if word.endswith('le'):
        count+=1
    if count == 0:
        count +=1
    return count
################################################
test_num_complex_word = []
count=0
for datum in test_processed_review_no_stem:
    count+=1
    if count % 1000 == 0:
        print count
    test_num_complex_word.append(sum([1 for w in datum if syllables(w)>=3]))

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000


In [38]:
# Readability
### Evaluating content quality and helpfulness of online product reviews: The interplay of review helpfulness vs. review content

# The Gunning-Fog Index (FOG)
test_FOG=[]
for i in range(len(test_num_sentence)):
    if test_num_sentence[i]==0:
        test_num_sentence[i] += 1
for i in range(len(test_data)):
    if len(test_processed_review_no_stem[i])==0:
        test_FOG.append(0.0)
    else:
        test_FOG.append(0.4*((float(len(test_processed_review_no_stem[i]))/test_num_sentence[i])+float(test_num_complex_word[i]*100)/len(test_processed_review_no_stem[i]))) 

In [39]:
# The Flesch–Kincaid Reading Ease (FK)
test_FK = []
for i in range(len(test_num_sentence)):
    if test_num_sentence[i]==0:
        test_num_sentence[i] += 1
for i in range(len(test_data)):
    if len(test_processed_review_no_stem[i])==0:
        test_FK.append(0.0)
    else:
        test_FK.append(0.39*(float(len(test_processed_review_no_stem[i]))/test_num_sentence[i])+float(test_num_complex_word[i]*11.8)/len(test_processed_review_no_stem[i])-15.59)

In [40]:
# The Automated Readability Index (ARI)
test_ARI = []
for i in range(len(test_data)):
    if len(test_processed_review_no_stem[i])==0:
        test_ARI.append(0.0) 
    else:
        test_ARI.append(4.71*(float(test_num_character[i])/len(test_processed_review_no_stem[i]))+0.5*(float(len(test_processed_review_no_stem[i]))/test_num_sentence[i])-21.43)

In [41]:
# The Coleman-LiauIndex (CLI)
test_CLI = []
for i in range(len(test_data)):
    if len(test_processed_review_no_stem[i])==0:
        test_CLI.append(0.0) 
    else:
        test_CLI.append(5.89*(float(test_num_character[i])/len(test_processed_review_no_stem[i]))+0.3*(float(test_num_sentence[i])/len(test_processed_review_no_stem[i]))-15.8)

In [42]:
# Convert comparative and superlative to normal
test_processed_review_no_stem_normalized=[]
wnl = WordNetLemmatizer()
count=0
for datum in test_processed_review_no_stem:
    count+=1
    if count % 1000 ==0:
        print count
    tmp_word_list = []
    for word in datum:
        word = wnl.lemmatize(word, 'n')
        word = wnl.lemmatize(word, 'v')
        word = wnl.lemmatize(word, 'a')
        word = wnl.lemmatize(word, 's')
        word = wnl.lemmatize(word, 'r')
        tmp_word_list.append(word)
    test_processed_review_no_stem_normalized.append(tmp_word_list)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000


In [43]:
# Information Complexity
# For normalized word not in top-50
test_tag_count = []
num_types = len(word_types)
count=0
for datum in test_processed_review_no_stem_normalized:
    count+=1
    if count % 1000 ==0:
        print count
    tags = nltk.pos_tag(datum)
    tag_num_dict = {}
    for i in range(num_types):
        tag_num_dict[word_types[i]]=sum([1 for t in tags if t[1]==word_types[i] and not t[0] in words_normarlized_most_frequent])
    test_tag_count.append(tag_num_dict)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000


In [44]:
# fraction score
test_fraction_noun = []
test_fraction_adj = []
test_fraction_verb = []
for datum in test_tag_count:
    test_fraction_noun.append(float(datum['NN']+datum['NNS']+datum['NNP'])/test_num_sentence[i])
    test_fraction_adj.append(float(datum['JJ']+datum['JJS']+datum['JJR'])/test_num_sentence[i])
    test_fraction_verb.append(float(datum['VB']+datum['VBD']+datum['VBG']+datum['VBN']+datum['VBP']+datum['VBZ'])/test_num_sentence[i])

In [45]:
test_text_mining_feature = numpy.zeros((len(test_data),8))
index=0
#for each row
# b f_noun f_adj f_verb FOG FK ARI CLI
for datum in test_data:
    test_text_mining_feature[index]=[datum['helpful'][1],test_fraction_noun[index],test_fraction_adj[index],test_fraction_verb[index],test_FOG[index],test_FK[index],test_ARI[index],test_CLI[index]]
    index += 1
    if index % 100 == 0:
        print index

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [46]:
# output
import pickle
test_data_np_array = open('test_text_feature','wb')
pickle.dump(test_text_mining_feature,test_data_np_array)
test_data_np_array.close()

In [47]:
len(test_text_mining_feature)

45058

In [48]:
len(test_data)

45058

In [49]:
test_text_mining_feature[0]

array([  0.        ,   1.66666667,   0.33333333,   0.33333333,
        11.35384615,  -8.70461538,  12.96769231,  19.11      ])

##############################################################

In [22]:
# Information Complexity with STEM
tag_count_stem = []
num_types = len(word_types)
count=0
for datum in processed_review_words_stem:
    count+=1
    if count % 100 ==0:
        print count
    tags = nltk.pos_tag(datum)
    tag_num_dict = {}
    for i in range(num_types):
        tag_num_dict[word_types[i]]=sum([1 for t in tags if t[1]==word_types[i]])
    tag_count_stem.append(tag_num_dict)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [34]:
print tag_count[2]
print tag_count_stem[2]

{'PRP$': 0, 'VBG': 0, 'VBD': 1, '``': 0, 'VBN': 2, "''": 0, 'VBP': 0, 'WDT': 0, 'JJ': 1, 'WP': 0, 'VBZ': 1, 'DT': 0, 'RP': 0, 'NN': 2, 'FW': 0, 'POS': 0, 'TO': 0, 'PRP': 0, 'RB': 2, 'NNS': 1, 'NNP': 0, 'VB': 1, 'WRB': 0, 'CC': 0, 'PDT': 0, 'RBS': 0, 'RBR': 0, 'CD': 1, 'EX': 0, 'IN': 0, 'WP$': 0, 'MD': 1, 'NNPS': 0, 'JJS': 1, 'JJR': 0, 'SYM': 0, 'UH': 0}
{'PRP$': 0, 'VBG': 0, 'VBD': 1, '``': 0, 'VBN': 1, "''": 0, 'VBP': 0, 'WDT': 0, 'JJ': 1, 'WP': 0, 'VBZ': 0, 'DT': 0, 'RP': 0, 'NN': 2, 'FW': 0, 'POS': 0, 'TO': 0, 'PRP': 0, 'RB': 3, 'NNS': 0, 'NNP': 0, 'VB': 1, 'WRB': 0, 'CC': 0, 'PDT': 0, 'RBS': 0, 'RBR': 0, 'CD': 1, 'EX': 0, 'IN': 0, 'WP$': 0, 'MD': 1, 'NNPS': 0, 'JJS': 2, 'JJR': 1, 'SYM': 0, 'UH': 0}


In [25]:
print processed_review_no_stem[1]
print processed_review_words_stem[1]

['horrible', 'animation', 'overly', 'glossy', 'computer', 'generated', 'garbage', 'quotstoryquot', 'plain', 'awful', 'could', 'actually', 'feel', 'brain', 'shrinking', 'watched', 'mess', 'find', 'hard', 'believe', 'kids', 'age', 'enjoy', 'mindless', 'nonsense', 'anyone', 'else', 'would', 'voluntarily', 'watch', 'pokemessblech']
[u'horribl', u'anim', u'overli', u'glossi', u'comput', u'gener', u'garbag', 'quotstoryquot', 'plain', u'aw', 'could', u'actual', 'feel', 'brain', u'shrink', u'watch', u'mess', 'find', 'hard', u'believ', u'kid', 'age', 'enjoy', u'mindless', u'nonsens', u'anyon', u'els', 'would', u'voluntarili', 'watch', 'pokemessblech']


In [35]:
print stemmer.stem('big')
print stemmer.stem('high')

big
high


# For Tf-idf

In [35]:
# build corpus, sentence in list
corpus=[]
for words in processed_review_words:
    corpus.append(' '.join(w for w in words))

In [36]:
# look for 100 most frequent words
words_count={}
for text in processed_review_words:
    for word in text:
        if not words_count.has_key(word):
            words_count[word] = 0
        words_count[word] += 1

In [37]:
words_most_frequent = sorted(words_count.iteritems(), key=lambda d:d[1], reverse = True)

In [38]:
words_most_frequent = [word[0] for word in words_most_frequent[:100]]

In [39]:
print words_most_frequent

['film', 'movie', 'one', 'like', 'dvd', 'good', 'time', 'would', 'great', 'also', 'first', 'story', 'see', 'even', 'get', 'well', 'much', 'really', 'two', 'show', 'films', 'series', 'best', 'many', 'love', 'people', 'way', 'dont', 'new', 'make', 'life', 'set', 'never', 'movies', 'little', 'still', 'made', 'could', 'season', 'characters', 'character', 'think', 'know', 'years', 'watch', 'back', 'man', 'better', 'seen', 'scenes', 'ever', 'original', 'end', 'world', 'say', 'scene', 'version', 'another', 'find', 'bad', 'work', 'episodes', 'though', 'music', 'real', 'old', 'part', 'may', 'cast', 'something', 'look', 'makes', 'young', 'going', 'want', 'every', 'episode', 'doesnt', 'long', 'quite', 'family', 'watching', 'three', 'lot', 'take', 'shows', 'however', 'always', 'things', 'didnt', 'last', 'plot', 'acting', 'come', 'actually', 'bit', 'big', 'role', 'around', 'must']


In [40]:
# build numpy array used for count word appearance times
words_count_np = numpy.zeros((len(filtered_train_data),100))

In [41]:
for i in range(len(filtered_train_data)):
    for j in range(len(words_most_frequent)):
        words_count_np[i][j] = processed_review_words[i].count(words_most_frequent[j])

In [42]:
print words_count_np[0]

[  2.   2.   2.   0.   0.   0.   2.   3.   0.   3.   2.   0.   3.   2.   0.
   1.   6.   0.   2.   4.   1.   0.   3.   1.   0.   4.   1.   0.   2.   1.
   1.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   1.   1.   1.   4.
   1.   0.   4.   0.   1.   2.   6.   1.  11.   0.   0.   8.   0.   0.   0.
   2.   7.   0.   1.   0.   3.   1.   0.   0.   1.   2.   1.   1.   0.   1.
   0.   3.   0.   0.   1.   0.   0.   1.   0.   0.   3.   0.   0.   1.   0.
   0.   0.   0.   0.   0.   0.   1.   2.   0.   0.]


In [43]:
# build numpy array used for count word appearance boolean
words_bool_np = numpy.zeros((len(filtered_train_data),100))

In [44]:
for i in range(len(filtered_train_data)):
    for j in range(len(words_most_frequent)):
        if words_count_np[i][j] != 0:
            words_bool_np[i][j] = 1

In [45]:
print words_bool_np[0]

[ 1.  1.  1.  0.  0.  0.  1.  1.  0.  1.  1.  0.  1.  1.  0.  1.  1.  0.
  1.  1.  1.  0.  1.  1.  0.  1.  1.  0.  1.  1.  1.  0.  0.  1.  0.  0.
  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  0.  1.  0.  1.  1.  1.  1.  1.
  0.  0.  1.  0.  0.  0.  1.  1.  0.  1.  0.  1.  1.  0.  0.  1.  1.  1.
  1.  0.  1.  0.  1.  0.  0.  1.  0.  0.  1.  0.  0.  1.  0.  0.  1.  0.
  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.]


In [46]:
words_tf_np = numpy.zeros((len(filtered_train_data),100))

In [47]:
for i in range(len(filtered_train_data)):
    for j in range(len(words_most_frequent)):
        if sum(words_count_np[i]) != 0:
            words_tf_np[i][j] = float(words_count_np[i][j])/sum(words_count_np[i])

In [48]:
words_idf_np = numpy.zeros((len(filtered_train_data),100))

In [57]:
for i in range(len(filtered_train_data)):
    for j in range(len(words_most_frequent)):
            words_idf_np[i][j] = math.log(len(filtered_train_data)/(sum(words_bool_np[:][j]) + 1))

In [58]:
tf_idf_np = numpy.zeros((len(filtered_train_data),100))

In [59]:
for i in range(len(filtered_train_data)):
    tf_idf_np[i] = words_tf_np[i]*words_idf_np[i]

In [61]:
# verify result
print words_tf_np[0][0]
print words_idf_np[0][0]
print words_tf_np[0][0] * words_idf_np[0][0]
print tf_idf_np[0][0]

0.016393442623
6.12329377071
0.100381865094
0.100381865094


In [105]:
import pickle
dataarray = open('tf_idf_np','wb')
pickle.dump(tf_idf_np,dataarray)
dataarray.close()

In [56]:
print train_data[3]

{'reviewerID': 'ADLVFFE4VBT8', 'asin': 'B002KSA4F6', 'helpful': [1, 1], 'reviewText': "As someone who climbed a mountain or two and dived into some unspeakably beautiful oceanic worlds, watched dolphins at play in their own habitat and spent a few days walking through what we used to call 'jungles' I know that no television experience can possibly replace actually being there. I also know that no human on Earth can possibly be to all the places the BBC crews covered in these series. And, thanks to their hard work, passion and BBC's equipping with the latest in HD gear, what came out is as close to 'being there' as today's technology allows it.'Life' kept me and the rest of the family captivated often in awe for the four days we spent watching the series, limiting ourselves to one disc per evening to allow ourselves sufficient time to talk about what we were watching and to go back from time to time and replay some of the more stunning takes. I'm not going to say what 'Life' is about be

### Old Code

In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
for datum in train_data:
    if datum['helpful'][1] != 0 and float(datum['helpful'][0])/datum['helpful'][1] > 1:
        print datum['helpful']
        train_data.remove(datum)
for datum in test_data:
    if datum['helpful'][1] != 0 and float(datum['helpful'][0])/datum['helpful'][1] > 1:
        print datum['helpful']
        test_data.remove(datum)

In [16]:
processed_review_words = []
stop_words = set(stopwords.words('english'))

for datum in train_data:
    ### Ignore capitalization and remove punctuation
    wordCount = defaultdict(int)
    punctuation = set(string.punctuation)
    r = ''.join([c for c in datum['reviewText'].lower() if not c in punctuation])
    ### Remove Number
    r_no_number = re.sub('[0-9]+', '', r)
    word_tokens = word_tokenize(r_no_number)
    ### Remove stop words
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    for word in filtered_sentence:
        if len(word) < 3:
            filtered_sentence.remove(word)
    processed_review_words.append(filtered_sentence)

In [17]:
print processed_review_words[2]

['great', 'show', 'sexcellent', 'transfersall', 'good', 'product']


In [18]:
print ' '.join(word for word in processed_review_words[2])

great show sexcellent transfersall good product


In [19]:
corpus=[]
for words in processed_review_words:
    corpus.append(' '.join(w for w in words))

In [13]:
#corpus=['This is a good one','This is not a good noe','This is a excellent one']

In [5]:
vectorizer=CountVectorizer()#该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频 
transformer=TfidfTransformer()#该类会统计每个词语的tf-idf权值
matrix_frequency = vectorizer.fit_transform(corpus)
tfidf=transformer.fit_transform(matrix_frequency)#第一个fit_transform是计算tf-idf，第二个fit_transform是将文本转为词频矩阵 
word=vectorizer.get_feature_names()#获取词袋模型中的所有词语weight=tfidf.toarray()#将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重

In [7]:
weight=tfidf.toarray()#将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重
for i in range(len(weight)):#打印每类文本的tf-idf词语权重，第一个for遍历所有文本，第二个for便利某一类文本下的词语权重
    print u"-------这里输出第",i,u"类文本的词语tf-idf权重------"
    for j in range(len(word)):
        print word[j],weight[i][j]

-------这里输出第 0 类文本的词语tf-idf权重------
excellent 0.0
good 0.558477835371
is 0.433707859509
noe 0.0
not 0.0
one 0.558477835371
this 0.433707859509
-------这里输出第 1 类文本的词语tf-idf权重------
excellent 0.0
good 0.420182921489
is 0.326309521953
noe 0.552490047084
not 0.552490047084
one 0.0
this 0.326309521953
-------这里输出第 2 类文本的词语tf-idf权重------
excellent 0.662839982347
good 0.0
is 0.391483971363
noe 0.0
not 0.0
one 0.504106891576
this 0.391483971363


In [ ]:
print matrix_frequency.toarray()

In [20]:
import copy
import nltk
import math
import string

from nltk.corpus import stopwords
from collections import Counter
from nltk.stem.porter import*

In [21]:
def get_tokens(text):
    #lower = text.lower()
    #remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
    #no_punctuation = lower.translate(remove_punctuation_map)
    #tokens = nltk.word_tokenize(no_punctuation)
    tokens = nltk.word_tokenize(text)
    return tokens

In [22]:
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

In [23]:
def tf(word, count):
    return float(count[word]) / sum(count.values())
def n_containing(word, count_list):
    return sum(1 for count in count_list if word in count)
def idf(word, count_list):
    return math.log(len(count_list)) / (1 + n_containing(word, count_list))
def tfidf(word, count, count_list):
    return tf(word, count) * idf(word, count_list)

In [24]:
def count_term(text):
    tokens = get_tokens(text)
    #filtered = [w for w in tokens if not w in stopwords.words('english')]
    stemmer = PorterStemmer()
    stemmed = stem_tokens(tokens, stemmer)
    count = Counter(stemmed)
    return count

In [25]:
#corpus=["asas is a","This is a good one dsccs dfsdfs dsfsdfsdf","dsadsav effeffefefeef","aaeaeaea ae aaea ae ae"]
#print len(corpus)

In [26]:
texts = copy.deepcopy(corpus)
countlist = []
count_num =0
for text in texts:
    count_num +=1
    if count_num % 1000 ==0:
        print count_num
    countlist.append(count_term(text))

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [ ]:
countlist

In [39]:
review_tf_idf=[]
num=0
for i, count in enumerate(countlist):
    num+=1
    if num % 1000 ==0:
        print num
    print("Top words in document {}".format(i + 1))
    scores = {word: tfidf(word, count, countlist) for word in count}
    review_tf_idf.append(scores)
    #sorted_words = sorted(scores.items(), key = lambda x: x[1], reverse=True)
    #for word, score in sorted_words:
    #    print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))

Top words in document 1
Top words in document 2
Top words in document 3
Top words in document 4
Top words in document 5
Top words in document 6
Top words in document 7
Top words in document 8
Top words in document 9
Top words in document 10
Top words in document 11
Top words in document 12
Top words in document 13
Top words in document 14
Top words in document 15
Top words in document 16
Top words in document 17
Top words in document 18
Top words in document 19
Top words in document 20
Top words in document 21
Top words in document 22
Top words in document 23
Top words in document 24
Top words in document 25
Top words in document 26
Top words in document 27
Top words in document 28
Top words in document 29
Top words in document 30
Top words in document 31
Top words in document 32
Top words in document 33
Top words in document 34
Top words in document 35
Top words in document 36
Top words in document 37
Top words in document 38
Top words in document 39
Top words in document 40
Top words

KeyboardInterrupt: 

In [46]:
words_idf={}
num=0
for i, count in enumerate(countlist):
    num+=1
    print num
    for word in count:
        if not words_idf.has_key(word):
            words_idf[word]=idf(word,countlist)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


KeyboardInterrupt: 